In [ ]:
%load_ext autoreload
%autoreload 2
import os
import matplotlib.pyplot as plt
import seaborn as sns
from os.path import join
from tqdm import tqdm
import pandas as pd
import pickle as pkl
import notebook_helper
import sys
from mprompt.modules import fmri
import seaborn as sns
from mprompt.data.data import TASKS_D3
import mprompt.data.data
import numpy as np
from scipy.special import softmax
from collections import defaultdict

import imodelsx.util
import pickle as pkl
from os.path import dirname, join
from typing import List
import torch.cuda
import os.path
import torch
import numpy.random
from transformers import AutoModelForCausalLM, AutoTokenizer
import joblib
from huth.utils_ds import make_word_ds

modules_dir = '/home/chansingh/mprompt/mprompt/modules'
SAVE_DIR_FMRI = join(modules_dir, 'fmri')

In [52]:
TEST_STORIES = ['wheretheressmoke', 'onapproachtopluto', 'fromboyhoodtofatherhood']
out = defaultdict(dict)
for subject in tqdm(['UTS01', 'UTS02', 'UTS03']):
    voxel_idxs = joblib.load(join(SAVE_DIR_FMRI, 'voxel_lists', f'{subject}_voxel_selectivity.jbl'))[:500]
    grids = joblib.load(join(SAVE_DIR_FMRI, 'stories', 'grids_all.jbl'))
    trfiles = joblib.load(join(SAVE_DIR_FMRI, 'stories', 'trfiles_all.jbl'))
    wordseqs = make_word_ds(grids, trfiles)    

    # loop over stories
    running_words = {}
    for k in TEST_STORIES:
        # get words from last 4 TRs, so -2 sec, -4 sec, -6 sec, -8 sec
        wordseq = wordseqs[k]
        words = np.array(wordseq.data)
        tr_times = wordseq.tr_times[10:-5]
        num_delays = 4
        running_words[k] = []
        for i in range(len(tr_times)):
            tr_time_max = tr_times[max(0, i - 1)]
            tr_time_min = tr_times[max(0, i - num_delays)]
            valid_times = (tr_time_min <= wordseq.data_times) & (wordseq.data_times <= tr_time_max)
            running_words[k].append(' '.join(words[valid_times]))

    # get resp
    resp = joblib.load(join(SAVE_DIR_FMRI, 'responses', f'{subject}_responses.jbl')) # these are already normalized
    resp = {k: resp[k][:, voxel_idxs] for k in TEST_STORIES} # narrow down the stories/voxels

    out[subject]['words'] = sum(list(running_words.values()), [])
    out[subject]['resp'] = np.concatenate(list(resp.values()))
    assert len(out[subject]['words']) == out[subject]['resp'].shape[0]
joblib.dump(out, join(SAVE_DIR_FMRI, 'stories', 'running_words.jbl'))

100%|██████████| 3/3 [00:34<00:00, 11.52s/it]


['/home/chansingh/mprompt/mprompt/modules/fmri/stories/running_words.jbl']